In [ ]:
import os, time
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

In [ ]:
roth_lunch = """Salad Bar Fruit, Salad Bar Veggies & Legumes, Salad Bar Dairy/Egg/Cheese, 
Salad Bar Meats & Tofu, Salad Bar Composed, Salad Bar Dressings, Pho Base, Pho Broth, Pho Main, 
Pho Toppings, Rice Rice Baby Base, Rice Rice Baby Main, Rice Rice Baby Veggies, Rice Rice Baby Topping
Rice Rice Baby Sauces, Swaadisht Main, Swaadisht Side, Pickled Bar, Soup, Beverages, Fruits, Self-Serve"""

class DiningScraper:
    def __init__(self, url):
        self.url = url
        
        chrome_options = Options()
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")
        # chrome_options.add_argument("--headless") # Keep off for debugging
        
        service = Service()
        self.driver = webdriver.Chrome(service=service, options=chrome_options)
        self.driver.get(url)

    # Closes disclaimer
    def close_disclaimer(self):
        try:
            # Check if the disclaimer modal is visible
            WebDriverWait(self.driver, 3).until(
                EC.visibility_of_element_located((By.ID, "cbo_nn_mobileDisclaimer"))
            )
            print("Disclaimer detected. Attempting to close...")
            
            # Force hide the modal via JS
            self.driver.execute_script(
                "document.getElementById('cbo_nn_mobileDisclaimer').style.display = 'none';"
            )
            # Remove the dark background overlay if it exists
            self.driver.execute_script(
                "var backdrop = document.getElementsByClassName('modal-backdrop')[0]; if(backdrop) backdrop.parentNode.removeChild(backdrop);"
            )
            print("Disclaimer closed.")
            time.sleep(1)
        except:
            print("No blocking disclaimer found (or already closed).")

    def scrape(self):
        output_data = []

        # Wait for page to load
        try:
            WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located((By.ID, "cbo_nn_unitDataList"))
            )
        except:
            print("Error: Page content didn't load.")
            return

        self.close_disclaimer()

        # Get dining options
        units = self.driver.find_elements(By.CSS_SELECTOR, "div.card.unit")
        unit_count = len(units)
        print(f"Found {unit_count} dining locations.")

        for i in range(unit_count):
            try:
                current_unit = units[i]

                # Scroll to make unit clickable
                self.driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", current_unit)
                time.sleep(0.5)
                
                # Extract name and status (open/closed)
                name = current_unit.find_element(By.CSS_SELECTOR, ".unit__name-link").text
                status = current_unit.find_element(By.CSS_SELECTOR, ".badge").text

                '''# Skip if closed or if name is empty (glitch protection)
                if not name: 
                    continue'''
                
                # Print current unit/status
                print(f"Checking: {name} ({status})")

                # Skip if closed; Roth page is different from others and menu doesn't change... use heuristic
                if "Closed" in status:
                    output_data.append(f"NAME: {name}\nSTATUS: {status}\n" + "-"*30)
                    continue
                elif name == "Rothschild Dining Center - Contains Peanuts & Treenuts":
                    output_data.append(f"NAME: {name}\nSTATUS: {status}\nMEAL: Lunch\nMENU: {roth_lunch}\n" + "-"*30)
                    continue

                # Click the unit name
                link = current_unit.find_element(By.CSS_SELECTOR, ".unit__name-link")
                self.driver.execute_script("arguments[0].click();", link)

                # Track meal name, default is for munchie marts
                meal_name = "General / All Day" 
                
                # Wait for either a meal link (Dining Hall) or a food item (Munchie Mart)
                try:
                    WebDriverWait(self.driver, 5).until(
                        lambda d: d.find_elements(By.CLASS_NAME, "cbo_nn_menuLink") or 
                                  d.find_elements(By.CLASS_NAME, "cbo_nn_itemHover")
                    )
                except:
                    output_data.append(f"NAME: {name}\nSTATUS: {status}\nMEAL: None\nMENU: None available\n" + "-"*30)
                    print("  -> No menu available.")
                    continue

                # Get links for meals (breakfast/lunch/dinner); Empty => munchie mart
                meal_links = self.driver.find_elements(By.CLASS_NAME, "cbo_nn_menuLink")
                
                if meal_links:
                    # Click a meal link (Check all for current day TBD)
                    first_meal = meal_links[0]
                    meal_name = first_meal.text
                    print(f"  -> Dining Hall detected. Clicking menu: {meal_name}")
                    self.driver.execute_script("arguments[0].click();", first_meal)
                    
                    # Wait for the next page (food items) to load
                    WebDriverWait(self.driver, 5).until(
                        EC.presence_of_element_located((By.CLASS_NAME, "cbo_nn_itemHover"))
                    )
                else:
                    print("  -> Direct Inventory detected.")

                '''# --- STEP 3: SCRAPE FOOD ITEMS ---
                # Wait for the item rows to appear
                WebDriverWait(self.driver, 5).until(
                    EC.presence_of_element_located((By.CLASS_NAME, "cbo_nn_itemHover"))
                )'''

                # Make soup
                soup = BeautifulSoup(self.driver.page_source, 'html.parser')
                menu_text = []
                current_category = "General"
                
                # Find ALL rows in the table (both headers and items)
                # We look for the main table body rows
                rows = soup.select('table.table tbody tr')
                
                category_items = [] # temporary storage for items in current category

                for row in rows:
                    classes = row.get('class', [])
                    
                    # If we're at a table header (categories), save category and start new one
                    if 'cbo_nn_itemGroupRow' in classes:
                        # If we have collected items for the previous category, save them now
                        if category_items:
                             menu_text.append(f"{current_category}: {', '.join(category_items)}")
                             category_items = [] # Reset
                        
                        # Set the new category name
                        current_category = row.text.strip()
                    # Otherwise, add new item to category
                    elif row.select_one('.cbo_nn_itemHover'):
                        item_name = row.select_one('.cbo_nn_itemHover').text.strip()
                        category_items.append(item_name)

                # Save last category
                if category_items:
                    menu_text.append(f"{current_category}: {', '.join(category_items)}")

                # Add data for current dining option to output
                output_data.append(f"NAME: {name}\nSTATUS: {status}\nMEAL: {meal_name}\nMENU: {menu_text}\n" + "-"*30)

            except Exception as e:
                print(f"Error checking index {i}: {e}")
                continue

        self.driver.quit()
        return "\n".join(output_data)

url = 'https://netnutrition.cbord.com/nn-prod/vucampusdining'
scraper = DiningScraper(url)
output = scraper.scrape()

In [ ]:
display(Markdown(output))

In [ ]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")

MODEL_GPT = 'gpt-4o-mini'
openai = OpenAI()

In [ ]:
system_prompt = f'''You are a helpful assistant for Vanderbilt students who don't know where to eat. 
Give suggestions and important information about where you suggest. Some important information may be:
- Munchie marts are for snacks
- The Residential Colleges are: Kissam, E. Bronson Ingram (EBI), Rothschild (Roth), Carmichael, Zeppos
- Residential Colleges are close to each other
- Commons is close to Village Apartments
- Commons and Residential Colleges are far apart
- Students can order ahead online with the Transact Mobile Ordering app. This is especially important for students
with eating accomodations (e.g. gluten free) and for VandyBlenz, as the line can get long

In your response, suggest only one or two places to eat. Then, only give relevant information about what you
suggested. Summarize the food options there, don't list all the items available.

Here is information of what food is currently available:
{output}
'''

In [ ]:
stream = openai.chat.completions.create(
    model = MODEL_GPT,
    messages = [
        {'role': 'system', 'content': system_prompt},
        {'role': 'user', 'content': "I'm hungry. I live in village. Where do you think I should eat? Also, whats available there now?"}
    ],
    stream = True
)

In [ ]:
response = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    response += chunk.choices[0].delta.content or ''
    update_display(Markdown(response), display_id=display_handle.display_id)